In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torchtext
import torchtext.vocab as vocab
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

In [3]:
device = torch.device("cuda:4") # TODO change to "cuda"

In [4]:
# Prepare the dataset
# Read the CSV file
data = pd.read_csv('data.csv')

# Splitting data into features and labels
X = data['tweet'].values

# Splitting the dataset into training and validation sets
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Creating DataFrame for training and validation sets
train_data = pd.DataFrame({'tweet': X_train})
test_data = pd.DataFrame({'tweet': X_test})

In [5]:
train_data

,tweet
0,RT @FunSizedYogi: @TheBlackVoice well how else...
1,Funny thing is....it's not just the people doi...
2,"RT @winkSOSA: ""@AintShitSweet__: ""@Rakwon_OGOD..."
3,@Jbrendaro30 @ZGabrail @ramsin1995 @GabeEli8 @...
4,S/o that real bitch
...,...
19821,The last at-bat at Yankee Stadium. Thanks for ...
19822,@_bradleey LMFAOOOO yooo I lost my elevator pa...
19823,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Ana..."
19824,RT @JennyJohnsonHi5: Just when I thought Justi...


In [6]:
test_data

,tweet
0,934 8616\ni got a missed call from yo bitch
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...
2,RT @eanahS__: @1inkkofrosess lol my credit ain...
3,RT @Maxin_Betha Wipe the cum out of them faggo...
4,Niggas cheat on they bitch and don't expect no...
...,...
4952,@GrizzboAdams @wyattnuckels haha ight nig calm...
4953,When you see kids being bad &amp; their parent...
4954,This bitch done blew my high
4955,Fat Trel that niggah &#128076;


In [7]:
# Target Model

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
# model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
# # This one said non-hate to a lot of hate speech as far as I have tried.

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")
target_model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english").to(device)

In [8]:
# For validation w.r.t. the target model (but does it technically increases the number of queries?)
def get_label(input_text):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Query the target model
    with torch.no_grad():
        target_outputs = target_model(**inputs)

    target_labels = target_outputs.logits.softmax(dim=1).tolist()[0]
    return target_labels

test_data['label'] = test_data['tweet'].apply(get_label) # ~ 2/3 mins
test_data

,tweet,label
0,934 8616\ni got a missed call from yo bitch,"[0.5540313124656677, 0.4459686875343323]"
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,"[0.37748023867607117, 0.6225197315216064]"
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,"[0.9582731127738953, 0.041726816445589066]"
3,RT @Maxin_Betha Wipe the cum out of them faggo...,"[0.095273457467556, 0.904726505279541]"
4,Niggas cheat on they bitch and don't expect no...,"[0.11824338138103485, 0.881756603717804]"
...,...,...
4952,@GrizzboAdams @wyattnuckels haha ight nig calm...,"[0.09661741554737091, 0.9033825397491455]"
4953,When you see kids being bad &amp; their parent...,"[0.3516906201839447, 0.6483093500137329]"
4954,This bitch done blew my high,"[0.21178992092609406, 0.7882100939750671]"
4955,Fat Trel that niggah &#128076;,"[0.08707571029663086, 0.9129243493080139]"


In [9]:
# Pre-trained embeddings
# Load pre-trained GloVe embeddings
embed_dim = 100
glove = vocab.GloVe(name='6B', dim=embed_dim)

# Get the vocabulary from the pre-trained embeddings
glove_vocab = glove.stoi  # Dictionary mapping words to their indices

In [10]:
# Clone Model class
class HateSpeechGRU(nn.Module):
    def __init__(self, pretrained_embeddings, hidden_size, output_dim, dropout):
        super(HateSpeechGRU, self).__init__()
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        self.gru = nn.GRU(embed_dim, hidden_size, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, output_dim)  # * 2 for bidirectional
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.embedding(text)  # text: [batch size, sent len]
        output, hidden = self.gru(embedded)  # output: [batch size, sent len, hidden_size * num_directions]
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)  # concatenate the final forward and backward hidden states
        hidden = self.dropout(hidden)
        output = self.fc(hidden)  # output: [batch size, output dim]
        return output

In [11]:
# Clone Model
# Define hyperparameters
pretrained_embeddings = glove.vectors # Create a matrix of pre-trained embeddings
hidden_size = 128  # Size of hidden states in the GRU
output_dim = 2  # Number of output classes (binary classification)
dropout = 0.5  # Dropout probability

clone_model = HateSpeechGRU(pretrained_embeddings, hidden_size, output_dim, dropout).to(device)

In [63]:
# Load pre-trained Sentence Transformer model
sent_transformer = SentenceTransformer("all-MiniLM-L6-v2").to(device)

# Create the embeddings for the training data
# ~ 30 secs
train_embeddings = sent_transformer.encode(train_data['tweet'].tolist(), convert_to_tensor=True).tolist()

#normalize the embeddings
train_embeddings = torch.tensor(train_embeddings).to(device)
train_embeddings = (train_embeddings - torch.min(train_embeddings, dim=0).values) / (torch.max(train_embeddings, dim=0).values - torch.min(train_embeddings, dim=0).values)
print(train_embeddings.shape)
# print(train_embeddings.shape)

# train_data['embedding'] = train_embeddings.cpu().tolist()

train_data['embedding'] = train_embeddings
train_data

,tweet,embedding
0,RT @FunSizedYogi: @TheBlackVoice well how else...,"[-0.04517167806625366, 0.13797102868556976, 0...."
1,Funny thing is....it's not just the people doi...,"[0.05024722218513489, 0.0015787003794685006, 0..."
2,"RT @winkSOSA: ""@AintShitSweet__: ""@Rakwon_OGOD...","[-0.11634157598018646, 0.04812651500105858, 0...."
3,@Jbrendaro30 @ZGabrail @ramsin1995 @GabeEli8 @...,"[-0.15736794471740723, -0.020282883197069168, ..."
4,S/o that real bitch,"[-0.11650463938713074, -0.026171239092946053, ..."
...,...,...
19821,The last at-bat at Yankee Stadium. Thanks for ...,"[-0.027169346809387207, 0.11435814201831818, 0..."
19822,@_bradleey LMFAOOOO yooo I lost my elevator pa...,"[-0.028217773884534836, -0.05430904030799866, ..."
19823,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Ana...","[0.020860467106103897, -0.0533132441341877, 0...."
19824,RT @JennyJohnsonHi5: Just when I thought Justi...,"[0.009069086983799934, -0.011038362048566341, ..."


In [79]:
# Load pre-trained Sentence Transformer model
sent_transformer = SentenceTransformer("all-MiniLM-L6-v2").to(device)

# Create the embeddings for the training data
# ~ 30 secs
train_embeddings = sent_transformer.encode(train_data['tweet'].tolist(), convert_to_tensor=True).tolist()

#normalize the embeddings
train_embeddings = torch.tensor(train_embeddings).to(device)
print(train_embeddings.shape)
train_embeddings = nn.functional.normalize(train_embeddings, p=2, dim=1)
print(train_embeddings.shape)

train_data['embedding'] = train_embeddings.cpu().tolist()
train_data

torch.Size([19826, 384])
torch.Size([19826, 384])


,tweet,embedding
0,RT @FunSizedYogi: @TheBlackVoice well how else...,"[-0.04517167806625366, 0.13797102868556976, 0...."
1,Funny thing is....it's not just the people doi...,"[0.05024722218513489, 0.0015787003794685006, 0..."
2,"RT @winkSOSA: ""@AintShitSweet__: ""@Rakwon_OGOD...","[-0.11634157598018646, 0.04812651500105858, 0...."
3,@Jbrendaro30 @ZGabrail @ramsin1995 @GabeEli8 @...,"[-0.15736794471740723, -0.020282883197069168, ..."
4,S/o that real bitch,"[-0.11650464683771133, -0.026171240955591202, ..."
...,...,...
19821,The last at-bat at Yankee Stadium. Thanks for ...,"[-0.027169346809387207, 0.11435814201831818, 0..."
19822,@_bradleey LMFAOOOO yooo I lost my elevator pa...,"[-0.028217773884534836, -0.05430904030799866, ..."
19823,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Ana...","[0.020860467106103897, -0.0533132441341877, 0...."
19824,RT @JennyJohnsonHi5: Just when I thought Justi...,"[0.009069086983799934, -0.011038362048566341, ..."


In [82]:
# Table to store the previous queries (for wise query selection later)
table = pd.DataFrame({
    'tweet': [],
    'embedding': [],
    't_out': [],
    'c_out': [],
    'count': []
})

alpha = 0.5 # Weight for 'dissimilariy with the previous queries' term from the formula
beta = 1-alpha # Weight for 'similarity with the previous queries that had hish disagreement' term from the formula


# Train the clone model
# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(clone_model.parameters(), lr=0.0001)

batch_size = 128

# Define number of epochs
epoch = 0

loss_vals = []

scaler = GradScaler()

# Training loop
    
while True:
    clone_model.train()
    
    if table.shape[0] == 0: # Cold start for the first query (can be random)
        idxs = torch.randperm(len(train_data))[:batch_size].tolist()
    else:
        # Wise query selection
        # Calculate the cosine similarity between the embeddings of the training data and the table
        similarities = util.cos_sim(torch.tensor(train_data['embedding'].tolist()), torch.tensor(table['embedding'].tolist()))
        
        # Calculate the average cosine similarity for each training data
        # 'dissimilariy with the previous queries'
        avg_similarities = similarities.max(dim=1).values

        # 'similarity with the previous queries that had hish disagreement'
        disagreement = torch.tensor((abs(table['c_out'] - table['t_out'])).tolist())
        
        similarities = similarities * disagreement
        avg_wrt_disagreement = similarities.mean(dim=1)

        # Calculating Z-scores for avg_similarities and avg_wrt_disagreement
        avg_similarities_z = (avg_similarities - avg_similarities.mean()) / avg_similarities.std()
        avg_wrt_disagreement_z = (avg_wrt_disagreement - avg_wrt_disagreement.mean()) / avg_wrt_disagreement.std()

        # Calculate the normalized formula for each training data
        formula = alpha * (-avg_similarities_z) + beta * avg_wrt_disagreement_z
        
        # Get the index of the training data with the lowest average cosine similarity
        idxs = formula.argsort(descending=True)[:batch_size].tolist()

        #print('argsorted formula', formula.sort(descending=True)[:batch_size]) 
    # else:
    #     # Wise query selection
    #     # Calculate the cosine similarity between the embeddings of the training data and the table
    #     similarities = util.cos_sim(torch.tensor(train_data['embedding'].tolist()), torch.tensor(table['embedding'].tolist()))
    #     # calculate the euclidian distance between embeddings
    #     # similarities = -torch.cdist(torch.tensor(train_data['embedding'].tolist()), torch.tensor(table['embedding'].tolist()), p=1)
    #     print('similarity shape', similarities.shape)
    #     # Calculate the average cosine similarity for each training data
    #     # 'dissimilariy with the previous queries'

    #     #count_vec = torch.tensor(table['count'].tolist())
    #     # raise count_vec to power of e
    #     #count_vec = torch.exp(count_vec)
    #     #avg_similarities = (similarities @ count_vec) / table.shape[0]
    #     #print('table shape', table.shape[0])


    #     #avg_similarities = similarities.mean(dim=1)

    #     # find the mean of the similarities with the previous 

    #     max_similarities = similarities.max(dim=1).values




    #     # 'similarity with the previous queries that had hish disagreement'
    #     disagreement = torch.tensor((abs(table['c_out'] - table['t_out'])).tolist())
        
    #     similarities = similarities * disagreement
    #     avg_wrt_disagreement = similarities.mean(dim=1)

    #     # Calculate the formula for each training data
    #     formula = alpha * (-max_similarities) + beta * avg_wrt_disagreement


        
    #     # Get the index of the training data with the lowest average cosine similarity
    #     idxs = formula.argsort(descending=True)[:batch_size].tolist()


    input_texts = train_data.iloc[idxs]['tweet']

    # Check if the input_texts are already queried
    will_added = [True] * len(input_texts)

    count = 0
    for t in range(len(input_texts)):
        if input_texts.iloc[t] in table['tweet'].tolist():
            will_added[t] = False
            count += 1
            table.loc[table['tweet'] == input_texts.iloc[t], 'count'] += 1
        #print(type(input_texts.iloc[t]))
    print('girdigi sayi', count)

    
            

    input_embeds = train_data.iloc[idxs]['embedding']

    # inputs = tokenizer(input_text, return_tensors="pt")
    input_tokens = tokenizer(input_texts.tolist(), return_tensors="pt", padding=True, truncation=False).to(device)

    # Query the target model
    with torch.no_grad():
        target_outputs = target_model(**input_tokens)

    target_labels = target_outputs.logits.softmax(dim=1).tolist()

    # Conver input_text to tensor using glove_vocab
    input_glove = [[glove_vocab[word] if word in glove_vocab else glove_vocab['unk'] for word in input_text.split()] for input_text in input_texts]
    max_len = max([len(input_text) for input_text in input_glove])
    input_glove = [input_text + [glove_vocab['unk']] * (max_len - len(input_text)) for input_text in input_glove]
    input_glove = torch.tensor(input_glove, dtype=torch.long).to(device)

    target_outputs = torch.tensor(target_labels).to(device)

    # Forward pass through the clone model
    with autocast():
        clone_logits = clone_model(input_glove)
        clone_outputs = torch.softmax(clone_logits, dim=-1)
        loss = criterion(clone_outputs, target_outputs)

    optimizer.zero_grad()

    scaler.scale(loss).backward()

    # Update model weights considering the scaled gradients
    scaler.step(optimizer)

    # Updates the scale for next iteration
    scaler.update()

    # Remove the queried data from the training data
    input_texts = input_texts[will_added]
    input_embeds = input_embeds[will_added]
    target_outputs = target_outputs[will_added]
    clone_outputs = clone_outputs[will_added]

    # Add row to table
    row = pd.DataFrame({'tweet': input_texts, 'embedding': input_embeds, 't_out': target_outputs[:,0].tolist(), 'c_out': clone_outputs[:,0].tolist(), 'count': [1] * len(input_texts)})
    table = pd.concat([table, row], ignore_index=True)

    # Evaluation
    clone_model.eval()

    with torch.inference_mode():

        # Updating the table
        input_glove_2 = [[glove_vocab[word] if word in glove_vocab else glove_vocab['unk'] for word in input_text.split()] for input_text in table['tweet'].tolist()]
        max_len = max([len(input_text) for input_text in input_glove_2])
        input_glove_2 = [input_text + [glove_vocab['unk']] * (max_len - len(input_text)) for input_text in input_glove_2]
        input_glove_2 = torch.tensor(input_glove_2, dtype=torch.long).to(device)

        with autocast():
            clone_logits_2 = clone_model(input_glove_2)
            clone_outputs_2 = torch.softmax(clone_logits_2, dim=-1)

        table['c_out'] = clone_outputs_2[:,0].tolist()
        
        # Validation set
        inputs = [[glove_vocab[word] if word in glove_vocab else glove_vocab['unk'] for word in txt.split()] for txt in test_data['tweet'].tolist()] # TODO: Do once, don't do it in every epoch
        max_len = max([len(txt) for txt in inputs])
        inputs = [txt + [glove_vocab['unk']] * (max_len - len(txt)) for txt in inputs]
        test_pred = clone_model(torch.tensor(inputs, device=device))
        test_loss = criterion(test_pred, torch.tensor(test_data['label'].tolist(), device=device))

    loss_vals.append(test_loss.item())
    print(f'Epoch [{epoch + 1}], Loss: {test_loss.item()}')

    epoch += 1

    print('table size ' ,table.shape)
    print('train_data size ' ,input_texts.shape)


girdigi sayi 0
Epoch [1], Loss: 3.887749433517456
table size  (128, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [2], Loss: 3.864238977432251
table size  (256, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [3], Loss: 3.8418831825256348
table size  (384, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [4], Loss: 3.820136785507202
table size  (512, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [5], Loss: 3.7993886470794678
table size  (640, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [6], Loss: 3.7764151096343994
table size  (768, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [7], Loss: 3.755317449569702
table size  (896, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [8], Loss: 3.7334978580474854
table size  (1024, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [9], Loss: 3.7133593559265137
table size  (1152, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [10], Loss: 3.69266939163208
table size  (1280, 5)
train_data size  (128,)
girdigi sayi 0
Epoch [11], Loss: 3.6716384

KeyboardInterrupt: 

In [70]:
# take the unique tweets of table
table2 = table.drop_duplicates(subset='tweet', keep='first')
table2

,tweet,embedding,t_out,c_out,count
0,"This is my ""sup bitches"" face http://t.co/mqPq...","[-0.1008186861872673, 0.006707882042974234, 0....",0.522070,0.999997,1.0
1,Two days gone she been gone so long you can't ...,"[-0.10198283940553665, -0.05068354308605194, 0...",0.214770,0.999991,1.0
2,#Yankees #FireCashman I don't want Arod back.,"[-0.025286220014095306, -0.03454281762242317, ...",0.368439,0.999994,1.0
3,&#8220;@lifeof_brandon: @100046729 we'll be to...,"[-0.13836750388145447, 0.005380598362535238, 0...",0.127643,0.999996,1.0
4,RT @ThatKidJony: I'm trying to pass my nigguh....,"[-0.09449616074562073, 0.05206477642059326, 0....",0.101626,0.999992,1.0
...,...,...,...,...,...
91,@ohVixen I love laying in bed by myself becaus...,"[0.0153201287612319, -0.0395226776599884, 0.03...",0.889367,0.999998,1.0
92,"#NP ""Centuries"" ~FOB","[-0.0013627943117171526, 0.09317202121019363, ...",0.662812,0.999993,1.0
93,yay I like when jacob has colored layouts,"[-0.10147412866353989, 0.006389786023646593, 0...",0.973427,0.999873,1.0
94,"I know how it feels to not have shit, believe ...","[0.07379258424043655, -0.02337886579334736, 0....",0.962512,0.999999,1.0
